<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Red%20convolucional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
# ARRANCAMOS CON LO BÁSICO
import numpy as np
import tensorflow as tf
import pandas as pd
import random
from google.colab import files

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


Version:  2.2.0
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [102]:
# CARGAMOS LOS CSV
url_train = 'https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'
url_test = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test.csv'

train_tmp_df = pd.read_csv(url_train)
test_df = pd.read_csv(url_test)
train_tmp_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [104]:
# LIMPIAMOS Y CHUSMEAMOS
train_tmp_df = train_tmp_df.drop(['keyword', 'location'], axis = 1)
train_tmp_df.head()
train_tmp_df.shape

(7613, 3)

In [105]:
# EXTRAEMOS UN SET DE VALIDACIÓN DEL 10%
from sklearn.model_selection import train_test_split

train_df, validation_df = train_test_split(train_tmp_df, test_size = 0.1)
train_df.shape[0]

6851

In [106]:
validation_df.shape[0]

762

In [107]:
# HACEMOS UN EMBEDDING PROVISTO POR UN "TensorFlow Hub model"

embedding_url = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding_url,
                           input_shape = [], 
                           dtype = tf.string,
                           trainable = True)

train_text_list = np.asarray(train_df['text'])
train_target_list = np.asarray(train_df['target'])

hub_layer(train_text_list[:3])



<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[-0.35029796, -0.35403475,  1.3561372 , -0.13495691, -1.0487702 ,
        -1.31472   , -0.9778609 ,  0.6905644 ,  0.46023297, -0.14615352,
        -0.34302536,  0.4875973 ,  0.7112628 , -0.21723983, -0.9430727 ,
         1.5455441 ,  1.0691601 , -0.22846644, -0.65636754, -0.11556711],
       [ 0.16114463, -1.6570708 ,  0.62960863, -0.48008874, -3.0408585 ,
        -0.8143956 , -0.8158874 , -0.30082893,  0.90850145, -0.5935487 ,
        -1.5110894 ,  0.47073632,  0.7127545 ,  0.6626567 , -2.2454321 ,
         0.829015  ,  2.6766765 , -0.06736254, -1.2845464 , -0.7452603 ],
       [ 1.1019081 , -1.3617902 ,  0.6159662 ,  1.9097294 , -1.2706358 ,
        -0.9123574 , -0.8715629 ,  1.224546  ,  1.1606469 , -0.16531052,
        -2.5867066 ,  1.3190113 ,  0.08228455,  0.05794315, -0.81064826,
         0.75463057,  0.93189734, -0.7646536 , -1.3636161 , -0.44924736]],
      dtype=float32)>

In [108]:
# ARMAMOS EL MODELO
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_10 (KerasLayer)  (None, 20)                400020    
_________________________________________________________________
dense_23 (Dense)             (None, 16)                336       
_________________________________________________________________
dense_24 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 17        
Total params: 400,645
Trainable params: 400,645
Non-trainable params: 0
_________________________________________________________________


In [109]:
# COMPILAMOS Y ENTRENAMOS

# NOTAS:
# Con 20 epochs obtiene mejor resultado que con 30

model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'])

history = model.fit(train_text_list, train_target_list, batch_size=512,
                    epochs = 20,
                    verbose = 1)

Epoch 1/20
14/14 [==============================] - 0s 12ms/step - loss: 0.7145 - accuracy: 0.5644
Epoch 2/20
14/14 [==============================] - 0s 11ms/step - loss: 0.6631 - accuracy: 0.5792
Epoch 3/20
14/14 [==============================] - 0s 11ms/step - loss: 0.6252 - accuracy: 0.6068
Epoch 4/20
14/14 [==============================] - 0s 13ms/step - loss: 0.5909 - accuracy: 0.6549
Epoch 5/20
14/14 [==============================] - 0s 12ms/step - loss: 0.5603 - accuracy: 0.7025
Epoch 6/20
14/14 [==============================] - 0s 11ms/step - loss: 0.5356 - accuracy: 0.7265
Epoch 7/20
14/14 [==============================] - 0s 11ms/step - loss: 0.5153 - accuracy: 0.7419
Epoch 8/20
14/14 [==============================] - 0s 11ms/step - loss: 0.4986 - accuracy: 0.7552
Epoch 9/20
14/14 [==============================] - 0s 11ms/step - loss: 0.4842 - accuracy: 0.7673
Epoch 10/20
14/14 [==============================] - 0s 11ms/step - loss: 0.4707 - accuracy: 0.7740
Epoch 11/

In [110]:
# EVALUAMOS CON EL SET DE VALIDACION
validation_text_list = np.asarray(train_df['text'])
validation_target_list = np.asarray(train_df['target'])

loss, precision = model.evaluate(validation_text_list, validation_target_list)
print('Precisión: %.2f' % (precision*100))

215/215 [==============================] - 0s 2ms/step - loss: 0.3385 - accuracy: 0.8479
Precisión: 84.79


In [111]:
# ARMAMOS LA PREDICCIÓN
test_text_list = np.asarray(test_df['text'])
prediction = model.predict_classes(test_text_list)
prediction

array([[1],
       [1],
       [1],
       ...,
       [0],
       [1],
       [0]], dtype=int32)

In [112]:
# EXPORTAMOS EL CSV EN EL FORMATO QUE PIDE KAGGLE
test_df['target'] = prediction
export = pd.DataFrame()
export['id'] = test_df['id']
export['target'] = test_df['target']
export = export.set_index('id')
export.to_csv('submit.csv')
files.download('submit.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>